**Machine Learning Tasks**
* Problem statement
* Read in Data
* Understand Data/Visualize Data
* Clean Data
* Preprocessing
    * Tokenization
    * Embedding
* Build Model
* Train
* Prediction 
* Problems with models

**Problem statement**

> "Can you help detect toxic comments ― and minimize unintended model bias? That's your challenge in this competition.
> 
> The Conversation AI team, a research initiative founded by Jigsaw and Google (both part of Alphabet), builds technology to protect voices in conversation. A main area of focus is machine learning models that can identify toxicity in online conversations, where toxicity is defined as anything rude, disrespectful or otherwise likely to make someone leave a discussion."

https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/overview

In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing import text, sequence
from gensim.models import KeyedVectors
import time

In [ ]:
# capture start time for metrics
start = time.time()
# show all columns when printing out tables
pd.options.display.max_columns = None


**Read in data**

> At the end of 2017 the Civil Comments platform shut down and chose make their ~2m public comments from their platform available in a lasting open archive so that researchers could understand and improve civility in online conversations for years to come. Jigsaw sponsored this effort and extended annotation of this data by human raters for various toxic conversational attributes.

https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/data

We can read in the input data files by storing them in a Pandas dataframes.
You can use the dataframe method 'head(n)' to view the first n rows of a dataframe

In [ ]:
train_df = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv', nrows=1000000)
#EXERCISE:Let's look at some rows of the data. You can see this with the pandas .head() function
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html

In [ ]:
test_df = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv', nrows=1000000)
#EXERCISE:Let's use the head function again to see how the test set differs

**Understand Data**




**Correlation** 

Relationship between each feature and target variable:
*     Positive Correlation: both variables change in the same direction.
*     Neutral Correlation: No relationship in the change of the variables.
*     Negative Correlation: variables change in opposite directions.

In [ ]:
cols_for_pairwise_correlation = ['severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat', 'asian', 'atheist', 'bisexual', 'black', 'buddhist', 'christian', 'female', 'heterosexual', 'hindu', 'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability', 'jewish', 'latino', 'male', 'muslim', 'other_disability', 'other_gender', 'other_race_or_ethnicity', 'other_religion', 'other_sexual_orientation', 'physical_disability', 'psychiatric_or_mental_illness', 'transgender', 'white', 'funny', 'wow', 'sad', 'likes', 'disagree', 'sexual_explicit', 'identity_annotator_count', 'toxicity_annotator_count']
target_col_for_pairwise_correlation = 'target'
pairwise_correlation = []

for col in cols_for_pairwise_correlation:
    # Compute pairwise correlation of columns, excluding NA/null values.
    corr = train_df[col].corr(train_df[target_col_for_pairwise_correlation])
    #print("Correlation between {} and {}: {:f}".format(target_col, col, corr))
    pairwise_correlation.append({'Score': corr, 'Feature': col})

correlation_df = pd.DataFrame(pairwise_correlation)
correlation_df.sort_values(by=['Score'], ascending=False)

Exploring the comment_text column 

In [ ]:
print("Average comment length:", train_df.comment_text.str.len().mean())
print("Max comment length:", train_df.comment_text.str.len().max())

**Distribution**

Visualize the distribution of the target variable using Kernel Density Estimate Plot.
In statistics, kernel density estimation (KDE) is a non-parametric way to estimate the probability density function (PDF) of a random variable. This function uses Gaussian kernels and includes automatic bandwidth determination.

In [ ]:
# Full data set takes a long time to plot 
train_df['target'].head(10000).plot.kde()

**Clean up data**

In [ ]:
# Let's select the data that we're going to use in our experiment

x_train = train_df['comment_text'].astype(str)
x_test = test_df['comment_text'].astype(str)
y_train = train_df['target'].values

y_aux_train = train_df[['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']].values

In [ ]:
# Convert values for most common identity labels to boolean
IDENTITY_COLUMNS = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness'
]
for column in IDENTITY_COLUMNS + ['target']:
    train_df[column] = np.where(train_df[column] >= 0.5, True, False) 
    
#EXERCISE:Use the head function again to see the updated dataframe


**Preprocessing**

Use tokenizer to create dictionary of words

In [ ]:

CHARS_TO_REMOVE = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n“”’\'∞θ÷α•à−β∅³π' 

# Let's first break down comment texts
tokenizer = text.Tokenizer(filters=CHARS_TO_REMOVE, lower=False)

#http://faroit.com/keras-docs/1.2.2/preprocessing/text/#tokenizer
#EXERCISE: call the tokenizer fit_on_texts method that fits on x_train and x_test, then display word_index dictionary for tokenizer
#what can you observe?

# fit_on_texts Updates internal vocabulary based on a list of texts. 
# It creates the vocabulary index based on word frequency, the lower value the more frequent the word is (value 0 is reserved for padding). 
# eg "The cat sat on the mat." => will create a dictionary s.t. word_index["the"] = 1; word_index["cat"] = 2 ..

#EXERCISE: display the dictionary of words created by tokenizer


In [ ]:
#EXERCISE:Use the tokenizer texts_to_sequences method to transform each comment_text to a sequence of integers. Each word in the comment should be replaced 
# with its corresponding integer value from the word_index dictionary. Then, display a sample comment
x_train = # 

In [ ]:
#EXERCISE do the same for the test data

x_test = # 


In [ ]:
# We will be processing comment vectors so it's useful to make them the same length.
# pad_sequences is used to ensure that all sequences in a list have the same length. 
# By default this is done by padding 0 in the beginning of each sequence 
MAX_LEN = 297
x_train = sequence.pad_sequences(x_train, maxlen=MAX_LEN)
#EXERCISE display the vector above

In [ ]:
#EXERCISE Call pad_sequences for the test data
x_test = #
#EXERCISE display the vector above

In [ ]:
# We converted comments to vectors of numbers 
#
# What about the meaning of words used in the comments?
# 
EMBEDDING_FILES = [
    '../input/gensim-embeddings-dataset/crawl-300d-2M.gensim',
    '../input/gensim-embeddings-dataset/glove.840B.300d.gensim'
]

# Lets load one of the embedding files to see what's there
embeddings = KeyedVectors.load('../input/gensim-embeddings-dataset/crawl-300d-2M.gensim', mmap='r')
#
# How are words represented in a numerical space?

#https://radimrehurek.com/gensim/models/keyedvectors.html
#EXERCISE Use word_vec method to display a word vector for a word 'apple'


In [ ]:
#EXERCISE What's the length of that vector? Use size property


In [ ]:
# What is the "distance" between two words?
#
w1 = embeddings.word_vec('king')
w2 = embeddings.word_vec('queen')

dist = np.linalg.norm(w1-w2)

print(dist)
#EXERCISE Try different words and rerun the cell to see their distances

In [ ]:
# Calculate 'king' - 'man' + 'woman' = ?

embeddings.most_similar(positive=['woman', 'king'], negative=['man'])

#EXERCISE try something different embeddings.most_similar(positive=['beans', 'stew', 'Texas'])

In [ ]:
# What else can we do using embeddings?
#
embeddings.doesnt_match("cat mouse rose dog".split())

#EXERCISE Try differnet set of words and doesnt_match method to check which word doesn't match the rest


In [ ]:
embeddings.similarity('cat', 'dog')

In [ ]:
# It indicates similarity of meaning of words not their spelling:
embeddings.similarity('cat', 'car')

In [ ]:
# Our model we're be building will require embedding information for the words used in the comments
# We will construct an embedding_matrix for the words in word_index
# using pre-trained embedding word vectors from resource in path
def build_matrix(word_index, path):
    # we've seen the embeddings already
    embedding_index = KeyedVectors.load(path, mmap='r')
    
    # embedding_matrix is a matrix of len(word_index)+1  x 300
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    
    # use word_index to get the index of each word 
    # and place its embedding in the matrix at that index
    for word, i in word_index.items():
        for candidate_word in [word, word.lower()]:
            if candidate_word in embedding_index:
                embedding_matrix[i] = embedding_index[candidate_word]
                break
    return embedding_matrix

# concatenate results for each type of embedding
embedding_matrix = np.concatenate(
    [build_matrix(tokenizer.word_index, f) for f in EMBEDDING_FILES], axis=-1)


In [ ]:
# To address the bias aspect we will create a set of weights that will reduce significance of samples containing identity words
# Build an array to provide a weight for each training sample. 
# (we'll indicating the weight for each of those samples during the training)
sample_weights = np.ones(len(x_train), dtype=np.float32)
sample_weights += train_df[IDENTITY_COLUMNS].sum(axis=1)
sample_weights += train_df['target'] * (~train_df[IDENTITY_COLUMNS]).sum(axis=1)
sample_weights += (~train_df['target']) * train_df[IDENTITY_COLUMNS].sum(axis=1) * 5
sample_weights /= sample_weights.mean()

**Building Our Model**

Now that we've preproccessed the data, it's time to build and train a model with the embeddings we created so that we can make predictions on comments.
In the below function, we create the different layers of the model using the Keras.layers libray.

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, add, concatenate
from keras.layers import CuDNNLSTM, Bidirectional, GlobalMaxPooling1D


#units parameters in Keras.layers.LSTM/cuDNNLSTM
#it it the dimension of the output vector of each LSTM cell.
LSTM_UNITS = 128
DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS

# embedding_matrix: The matrix we created in the preprocessing stage
# num_aux_targets: The number of auxiliary target variables we have.
def build_model(embedding_matrix, num_aux_targets):
    words = Input(shape=(None,))
    
    #The Keras Embedding will turn our matrix of indexes into dense vectors of fixed size
    #The first parameter represents our max index integer 
    x = Embedding(*embedding_matrix.shape, weights=[embedding_matrix], trainable=False)(words)
    #Randomlly drop 0.2% of input values to help prevent overfitting (This prevents units from coadapting too much)
    x = SpatialDropout1D(0.2)(x)
    
    #CuDNNLSTM is a LSTM implementation using the Nvidia cuDNN library 
    #The LSTM units is the dimension of the output vector of each LSTM cell
    x = Bidirectional(CuDNNLSTM(LSTM_UNITS, return_sequences=True))(x)
    x = Bidirectional(CuDNNLSTM(LSTM_UNITS, return_sequences=True))(x)

    hidden = concatenate([
        #Global max pooling has a pool size equal to the size of the input.
        GlobalMaxPooling1D()(x),
        GlobalMaxPooling1D()(x),
    ])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    result = Dense(1, activation='sigmoid')(hidden)
    aux_result = Dense(num_aux_targets, activation='sigmoid')(hidden)
    
    model = Model(inputs=words, outputs=[result, aux_result])
    model.compile(loss='binary_crossentropy', optimizer='adam')

    return model


Let's now call our build_model function:

In [ ]:
#EXCERCISE: Lets call the above method to build our model 
model = #


**Visualizing Our Model**

We have now constructed a model that we can fit with our data. Before we do this, we can visualize our model to get a better understanding of what we just did.

The Keras Model has a summary() function that shows the layers of the model we've created:

In [ ]:
#EXCERCISE: Use the .summary() function on the model we created above to help see a summary of the different layers in the model



We can see another representation of our model by using a Keras utility function called plot_model and passing in our model

In [ ]:
from keras.utils import plot_model

#EXCERCISE: Use the plot_model function with our model as a parameter for another visual 


You can see the above model prints out two output layers, why does this make sense for our model?

We can now train our model and do some prediction. In the below code, we loop over our data 4 times, training and predicting on it each time, and then taking the average of those predictions.

In [ ]:
#this is the number of training samples to put in the model each step
BATCH_SIZE = 512

EPOCHS = 4
checkpoint_predictions = []
weights = []
for global_epoch in range(EPOCHS):
    model.fit(
        x_train,
        [y_train, y_aux_train],
        batch_size=BATCH_SIZE,
        epochs=1,
        verbose=2,
        sample_weight=[sample_weights.values, np.ones_like(sample_weights)]
        )
    checkpoint_predictions.append(model.predict(x_test, batch_size=2048)[0].flatten())
    weights.append(2 ** global_epoch)

predictions = np.average(checkpoint_predictions, weights=weights, axis=0)

submission = pd.DataFrame.from_dict({
    'id': test_df.id,
    'prediction': predictions
})
submission.to_csv('submission.csv', index=False)
print(predictions)
end = time.time()
print("Total time taken:", end - start)

How do you think we did with our predictions? Let's compare our predictions with the original test comments and see if our predictions seem right.

Recall that test_df was our original dataframe with the comments. Try playing around with the below for loop to compare comments with our predictions. Do they make sense?

In [ ]:
##EXCERCISE: Write a for loop to loop over each entry in our predictions and explore our toxicity predictions
#For example, output the index of each prediction > 80 and compare it with the original text

for prediction in predictions:


#print(test_df.comment_text[index])